In [112]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [113]:
conn = sqlite3.connect('../db.sqlite3')

In [114]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto", conn)

In [115]:
df_raw

,id,kenteken,bouwjaar,kilometer_stand,vermogen,is_handgeschakeld,is_benzine,prijs,url,titel,upload_datum,bron,apk,merk,model
0,2197,96-ZKX-1,2012,148283.0,68.0,1.0,1,3750.0,https://www.marktplaats.nl/a/auto-s/peugeot/m1...,Zeer Mooie Peugeot 107 1.0 Access Accent 2012!,2020-06-01 09:20:00,marktplaats,2021-05-14,Peugeot,107
1,2198,79-XKN-1,2012,100472.0,60.0,1.0,1,5950.0,https://www.marktplaats.nl/a/auto-s/volkswagen...,Volkswagen UP! High Up! White 3drs 1e eigenaar...,2020-06-01 20:13:00,marktplaats,2021-02-05,Volkswagen,up!
2,2199,NJ-700-B,2016,97139.0,60.0,1.0,1,8995.0,https://www.marktplaats.nl/a/auto-s/volkswagen...,Volkswagen Polo 1.0 BlueMotion 2016 Zwart APK ...,2020-05-29 08:17:00,marktplaats,2022-05-25,Volkswagen,Polo
3,2200,3-ZBJ-68,2014,137180.0,69.0,1.0,1,6299.0,https://www.marktplaats.nl/a/auto-s/toyota/m15...,Toyota Aygo 1.0 VVT-i x-play NL Auto / Airco /...,2020-05-31 12:33:00,marktplaats,2020-12-29,Toyota,Aygo
4,2201,18-ZLG-2,2011,142000.0,340.0,0.0,1,26900.0,https://www.marktplaats.nl/a/auto-s/audi/m1558...,Audi Quattro RS3 340 pk TOP STAAT leer 5drs au...,2020-06-01 18:30:00,marktplaats,2020-11-18,Quattro,RS3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4157,6354,59-GT-NB,2001,257399.0,98.0,1.0,1,500.0,https://www.marktplaats.nl/a/auto-s/renault/m1...,Renault Clio 1.4-16V 98pk Ludo (bj 2001),2020-05-31 14:51:00,marktplaats,2020-12-16,Renault,Clio
4158,6355,97-ZL-HJ,2008,270000.0,75.0,1.0,1,2300.0,https://www.marktplaats.nl/a/auto-s/renault/m1...,Renault Clio III 1.2 16V 75 pk 3D 2008 Zwart,2020-05-31 14:50:00,marktplaats,2020-12-03,Renault,Clio
4159,6356,H-438-LD,2020,5.0,73.0,1.0,1,12245.0,https://www.marktplaats.nl/a/auto-s/peugeot/m1...,Peugeot 108 1.0 e-VTi Active | Nieuwe auto | V...,2020-05-31 14:48:00,marktplaats,2024-02-28,Peugeot,108
4160,6357,33-LG-LP,2003,223025.0,75.0,1.0,1,1250.0,https://www.marktplaats.nl/a/auto-s/peugeot/m1...,Peugeot 206 1.4 XS,2020-05-31 14:47:00,marktplaats,2021-05-14,Peugeot,206


In [116]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "upload_datum", "apk", "merk", "model"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld
count,4162.000000,4.138000e+03,4154.000000,4098.000000,4159.000000
mean,2009.705911,1.221337e+05,95.997352,7049.174719,0.840827
std,6.060419,7.253854e+04,43.346595,7096.828028,0.365881
min,1972.000000,1.000000e+00,39.000000,1.000000,0.000000
25%,2005.000000,6.714750e+04,69.000000,2000.000000,1.000000
50%,2010.000000,1.200380e+05,84.000000,4950.000000,1.000000
75%,2015.000000,1.719502e+05,110.000000,9400.000000,1.000000
max,2020.000000,1.489152e+06,551.000000,119500.000000,1.000000


In [117]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]
df.describe()

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld
count,3515.000000,3501.000000,3515.000000,3515.000000,3512.000000
mean,2010.871977,113754.329906,95.951067,7525.518065,0.846241
std,5.048101,64947.495577,39.464160,5883.580371,0.360769
min,2001.000000,1.000000,50.000000,1050.000000,0.000000
25%,2007.000000,61619.000000,69.000000,2950.000000,1.000000
50%,2011.000000,112145.000000,86.000000,5950.000000,1.000000
75%,2015.000000,160618.000000,111.000000,9990.000000,1.000000
max,2020.000000,346565.000000,457.000000,29999.000000,1.000000


In [118]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

# def parse_titel(titel):
#     cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
#     stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
#     return stripped

# df['titel'] = df['titel'].apply(parse_titel)

In [119]:
df['merk'] = df['merk'].str.lower()
df['model'] = df['model'].str.lower()

df = pd.concat([df, pd.get_dummies(df['merk'], prefix='merk')], axis=1)
df = pd.concat([df, pd.get_dummies(df['model'], prefix='model')], axis=1)
df = df.drop(['merk', 'model'], axis=1).dropna()
df

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,upload_datum,apk,merk_abarth,merk_alfa,merk_audi,...,model_v60,model_vectra,model_vel,model_volkswagen,model_wagon,model_xsara,model_yaris,model_ypsilon,model_yrv,model_zr
0,2012,148283.0,68.0,3750.0,1.0,2,345,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2012,100472.0,60.0,5950.0,1.0,2,247,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016,97139.0,60.0,8995.0,1.0,5,721,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2014,137180.0,69.0,6299.0,1.0,3,209,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2011,142000.0,340.0,26900.0,0.0,2,168,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154,2017,9993.0,146.0,17940.0,1.0,3,423,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4158,2008,270000.0,75.0,2300.0,1.0,3,183,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4159,2020,5.0,73.0,12245.0,1.0,3,1365,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4160,2003,223025.0,75.0,1250.0,1.0,3,345,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [121]:
df_train.to_pickle("./train_data.pkl")
df_test.to_pickle("./test_data.pkl")